<a href="https://colab.research.google.com/github/auliraa/threads-sentiment-sparknlp/blob/main/threads_App_Sentiment_Analysis_SparkNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Latar Belakang Masalah:**
Aplikasi "Threads" menerima lebih dari 37.000 ulasan dalam waktu singkat. Bagi tim Product Manager dan Developer, membaca puluhan ribu ulasan satu per satu untuk mengetahui apakah respons pengguna positif atau negatif adalah hal yang mustahil (inefisien waktu dan tenaga). Tanpa otomatisasi, isu kritis dari pengguna bisa terlewat.

**Solusi yang Ditawarkan:**
Mengimplementasikan Sentiment Analysis menggunakan SparkNLP. Sistem ini akan secara otomatis membaca teks ulasan pengguna dan mengklasifikasikannya ke dalam kategori: Positive, Negative, atau Neutral.

**Manfaat:**

Efisiensi Waktu: Mengolah 37.000 data hanya dalam hitungan menit/detik.

Prioritas Penanganan: Tim developer bisa langsung memfilter ulasan berlabel "Negative" untuk memperbaiki bug atau fitur yang dikeluhkan tanpa harus menyortir ulasan bagus.

Insight Cepat: Mendapatkan gambaran umum kepuasan pengguna secara instan.

**Target Pengguna (User):** Product Manager, Data Analyst, dan Tim Customer Support aplikasi Threads.

In [ ]:
!pip install spark-nlp
!pip install pyspark

Cell 2: Import Library & Inisialisasi Spark
Di sini kita memulai "mesin" Spark.

In [4]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = sparknlp.start()

print(f"Spark NLP version: {sparknlp.version()}")
print(f"Apache Spark version: {spark.version}")

Spark NLP version: 6.2.2
Apache Spark version: 3.5.1


memuat dataset 37000_reviews_of_thread_app.csv.


In [5]:
file_path = "37000_reviews_of_thread_app.csv"

df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

#lihat struktur data dan 5 baris pertama
print("Jumlah data:", df.count())
df.printSchema()
df.select("review_description", "rating").show(5, truncate=False)

Jumlah data: 37631
root
 |-- _c0: string (nullable = true)
 |-- source: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_description: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- developer_response: string (nullable = true)
 |-- developer_response_date: string (nullable = true)
 |-- appVersion: string (nullable = true)
 |-- laguage_code: string (nullable = true)
 |-- country_code: string (nullable = true)

+-------------------------------------------------------------------------------------------------------------------------------------+------+
|review_description                                                                                                                   |rating|
+----------------------------------------------------------------------------------

Cell 4: Membersihkan Data (Data Preparation)

In [6]:

clean_df = df.select("review_description", "rating").dropna()

print(f"Jumlah data setelah pembersihan: {clean_df.count()}")

Jumlah data setelah pembersihan: 36610


Cell 5: Membangun Pipeline SparkNLP
menggunakan model pre-trained bernama analyze_sentiment. Pipeline ini akan mengubah teks mentah -> menjadi dokumen -> menjadi token (kata per kata) -> lalu dicek sentimennya.

In [7]:
# 1. DocumentAssembler
document_assembler = DocumentAssembler() \
    .setInputCol("review_description") \
    .setOutputCol("document")

# 2. Universal Sentence Encoder
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

# 3. SentimentDLModel
sentiment_dl = SentimentDLModel.pretrained(name="sentimentdl_use_twitter", lang="en") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment")

pipeline = Pipeline(stages=[
    document_assembler,
    use,
    sentiment_dl
])

pipeline_model = pipeline.fit(clean_df)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


Cell 6: Menjalankan Prediksi (Inference)

In [8]:
# Transformasi data: Menerapkan model ke data
result = pipeline_model.transform(clean_df)

final_df = result.select(
    F.col("review_description"),
    F.col("rating"),
    F.explode("sentiment.result").alias("prediction") # Mengambil hasil prediksi (positive/negative)
)

# Tampilkan hasil
final_df.show(10, truncate=50)

+--------------------------------------------------+------+----------+
|                                review_description|rating|prediction|
+--------------------------------------------------+------+----------+
|                                              Good|     5|  positive|
|                              Weak copy of Twitter|     1|  negative|
|i wish threads have a save button for images an...|     3|  positive|
|                                           Love it|     5|  positive|
|                                          Very god|     5|  positive|
|                                              Nice|     1|  positive|
|                                              Vain|     1|  positive|
|                     Not satisfied why this app...|     1|  negative|
|People want to leave twitter for this waste of ...|     1|  negative|
|                        Copy of Twitter hahaha lol|     1|  positive|
+--------------------------------------------------+------+----------+
only s

Cell 7: Evaluasi & Validasi

Jika Rating 5, harusnya sentimen positive.

Jika Rating 1, harusnya sentimen negative.

In [9]:
# Mari kita lihat contoh apakah prediksi model masuk akal dibandingkan dengan Rating asli

print("--- Contoh Review Positif (Rating 5) ---")
final_df.filter(F.col("rating") == 5).show(5, truncate=50)

print("--- Contoh Review Negatif (Rating 1) ---")
final_df.filter(F.col("rating") == 1).show(5, truncate=50)

# Menghitung distribusi prediksi
print("Distribusi Sentimen yang diprediksi:")
final_df.groupBy("prediction").count().show()

--- Contoh Review Positif (Rating 5) ---
+------------------+------+----------+
|review_description|rating|prediction|
+------------------+------+----------+
|              Good|     5|  positive|
|           Love it|     5|  positive|
|          Very god|     5|  positive|
|      I love it 💞|     5|  positive|
|               Osm|     5|  positive|
+------------------+------+----------+
only showing top 5 rows

--- Contoh Review Negatif (Rating 1) ---
+--------------------------------------------------+------+----------+
|                                review_description|rating|prediction|
+--------------------------------------------------+------+----------+
|                              Weak copy of Twitter|     1|  negative|
|                                              Nice|     1|  positive|
|                                              Vain|     1|  positive|
|                     Not satisfied why this app...|     1|  negative|
|People want to leave twitter for this waste o


Berdasarkan hasil evaluasi model SparkNLP menggunakan pipeline pre-trained sentimentdl_use_twitter, dapat disimpulkan bahwa:

Penyelesaian Masalah:
Model mampu mengklasifikasikan ribuan teks menjadi sentimen Positive, Negative, atau Neutral.

Akurasi Prediksi:

Model menunjukkan performa yang sangat baik pada ulasan positif. Teks pendek dan slang (seperti "Osm") dapat dikenali dengan tepat.

Pada ulasan negatif, model berhasil menangkap keluhan eksplisit (contoh: "Weak copy of Twitter").

Ditemukan sedikit anomali pada ulasan ambigu. Contoh: Ulasan dengan teks "Nice" tetapi Rating 1 diklasifikasikan sebagai Positive. Hal ini wajar karena secara linguistik "Nice" adalah kata positif, dan kemungkinan besar ini adalah bentuk sarkasme pengguna atau kesalahan input rating oleh pengguna (user labeling noise), bukan kegagalan total model.

Manfaat Nyata: Dengan distribusi data yang didapat (Negative: ~13.000, Positive: ~21.000), tim Developer Threads sekarang dapat memprioritaskan untuk membaca 13.000 ulasan berlabel negative saja untuk mencari bug atau fitur yang kurang, tanpa harus menyisir 21.000 ulasan positif yang isinya hanya pujian.

Kesimpulan: Implementasi SparkNLP ini siap digunakan sebagai alat bantu pendukung keputusan (Decision Support System) bagi tim produk Threads.